In [ ]:
from tardis.workflows.v_inner_solver import InnerVelocitySolverWorkflow
from tardis.io.configuration.config_reader import Configuration

In [ ]:
config = Configuration.from_yaml('../tardis_example.yml')

This code modifies the TARDIS example configuration to include convergence information for the inner boundary velocity solver.
Note that the number of shells is increased and the starting velocity is reduced to provide more granularity and a wider search window, respectively.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

In [ ]:
config.montecarlo.convergence_strategy['v_inner_boundary'] = {
    'damping_constant' : 0.5,
    'threshold' : 0.01,
    'type' : 'damped'
    }

config.montecarlo.convergence_strategy.stop_if_converged = True
config.model.structure.velocity.start = 5000 * u.km/u.s # Larger window over which to search
config.model.structure.velocity.num = 50 # Increase number of shells

workflow = InnerVelocitySolverWorkflow(
    config, tau=2.0/3,
    mean_optical_depth="rosseland"
)

In [ ]:
workflow.run()

## Plot the spectrum

In [ ]:
spectrum = workflow.spectrum_solver.spectrum_real_packets
spectrum_virtual = workflow.spectrum_solver.spectrum_virtual_packets
spectrum_integrated = workflow.spectrum_solver.spectrum_integrated

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 6.5))

spectrum.plot(label="Normal packets")
spectrum_virtual.plot(label="Virtual packets")
spectrum_integrated.plot(label='Formal integral')

plt.xlim(500, 9000)
plt.title("TARDIS example model spectrum")
plt.xlabel(r"Wavelength [$\AA$]")
plt.ylabel(r"Luminosity density [erg/s/$\AA$]")
plt.legend()
plt.show()

## Plot the convergence process

In [ ]:
%matplotlib inline

# extract plasma states
t_rad = workflow.iterations_t_rad
t_inner = workflow.iterations_t_inner
w = workflow.iterations_w
taus = workflow.iterations_mean_optical_depth
v_inner = workflow.iterations_v_inner_boundary

# remove all nans, or rows where all values are nan
t_rad = t_rad[~np.all(np.isnan(t_rad), axis=1)]
t_inner = t_inner[~np.isnan(t_inner)]
w = w[~np.all(np.isnan(w), axis=1)]
taus = taus[~np.all(np.isnan(taus), axis=1)]  
v_inner = v_inner[~np.isnan(v_inner)]

# initialize figure
fig,axes =  plt.subplots(2,2,figsize=(12,8))
plt.subplots_adjust(wspace=0.4,hspace=0.3)

# get the raw velocity grid 
vel = workflow.simulation_state.geometry.v_inner

# pick a colormap for the iterations
cmap = plt.get_cmap('rainbow',taus.shape[0])

# plot v inner change
axes[0,0].plot(v_inner,marker="o",color="b")
axes[0][0].set_xlabel("Iterations", fontsize=14)
axes[0][0].set_ylabel("v_inner (cm/s)", fontsize=14)
axes[0][0].grid(alpha=0.3)
axes[0][0].tick_params(axis='y', colors='blue')
axes[0][0].yaxis.label.set_color('blue')
# plot t inner change in same subplot
ax = axes[0][0].twinx()
ax.plot(t_inner,marker="s",color="r")
ax.set_ylabel("t_inner (K)", fontsize=14)
ax.tick_params(axis='y', colors='red')
ax.yaxis.label.set_color('red')

# plot the tau change
for i, tau in enumerate(taus):
    axes[0][1].plot(vel[-len(tau):], tau, color=cmap(i/taus.shape[0]),label=f"itr {i+1}",alpha=0.7)
axes[0][1].scatter(workflow.simulation_state.v_inner_boundary.value, np.log(2.0 / 3.0), color="k",marker="o")
axes[0][1].axhline(np.log(2.0 / 3.0), color='black', linestyle='--')
axes[0][1].axvline(workflow.simulation_state.v_inner_boundary.value, color='k', linestyle='--')
axes[0][1].set_xlabel("Velocity (cm/s)", fontsize=14)
axes[0][1].set_ylabel("log(tau)", fontsize=14)
axes[0][1].grid(alpha=0.3)
axes[0][1].set_title("1 Loop of all together - without resetting plasma", fontsize=14)

# plot t_rad and t_rad change 
for i in range(len(taus)):
    axes[1][0].plot(vel[-len(t_rad[i]):], t_rad[i],color=cmap(i/taus.shape[0]),label=f"itr {i+1}",alpha=0.7)
    axes[1][1].plot(vel[-len(w[i]):], w[i],color=cmap(i/taus.shape[0]),label=f"itr {i+1}",alpha=0.7)
axes[1][0].set_xlabel("Velocity (cm/s)", fontsize=14)
axes[1][0].set_ylabel("t_rad", fontsize=14)
axes[1][0].grid(alpha=0.3)
axes[1][1].set_xlabel("Velocity (cm/s)", fontsize=14)
axes[1][1].set_ylabel("dilution_factor", fontsize=14)
axes[1][1].grid(alpha=0.3)
axes[1][-1].legend(loc="upper right",bbox_to_anchor=(1.3,1))